In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
from keras.models import load_model
import json
import random
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from flask import Flask, render_template, request
!pip install flask_ngrok
from flask_ngrok import run_with_ngrok
!pip install pyngrok
from pyngrok import ngrok
!ngrok authtoken 2NC3EXtxmntp8LkDULb36JGY3TV_6xL2xXDh8mHANMfssrUh7

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://d177-34-90-131-84.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [21/Mar/2023 15:43:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Mar/2023 15:43:03] "GET /static/styles/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Mar/2023 15:43:05] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Mar/2023 15:43:08] "GET /get?msg=Hi HTTP/1.1" 200 -


...

Hi


INFO:werkzeug:127.0.0.1 - - [21/Mar/2023 15:52:28] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Mar/2023 15:52:28] "GET /static/styles/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Mar/2023 15:52:37] "GET /get?msg=How%20to%20place%20a%20float HTTP/1.1" 200 -


How to place a float


INFO:werkzeug:127.0.0.1 - - [21/Mar/2023 15:55:10] "GET /get?msg=How%20to%20add%20affiliation HTTP/1.1" 200 -


How to add affiliation


In [ ]:
lemmatizer = WordNetLemmatizer()
rootFolderPath = '/content/drive/MyDrive/Simple-Python-Chatbot-master/Simple-Python-Chatbot-master/'
model = load_model('/content/drive/MyDrive/Simple-Python-Chatbot-master/Simple-Python-Chatbot-master/kriya_chatbot_model.h5')

intents = json.loads(open('/content/drive/MyDrive/Simple-Python-Chatbot-master/Simple-Python-Chatbot-master/intents_kriya.json').read())
words = pickle.load(open('/content/drive/MyDrive/Simple-Python-Chatbot-master/Simple-Python-Chatbot-master/words_kriya.pkl','rb'))
classes = pickle.load(open('/content/drive/MyDrive/Simple-Python-Chatbot-master/Simple-Python-Chatbot-master/classes_kriya.pkl','rb'))


In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


In [ ]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))


In [ ]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]),verbose = 0)[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list



In [ ]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result



In [ ]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res



In [ ]:
app = Flask(__name__, template_folder='/content/drive/MyDrive/Simple-Python-Chatbot-master/Simple-Python-Chatbot-master/templates')
app.static_folder = '/content/drive/MyDrive/Simple-Python-Chatbot-master/Simple-Python-Chatbot-master/static'

run_with_ngrok(app)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    print(userText)
    # response = chatbot_response(userText)
    return chatbot_response(userText)
    
if __name__ == "__main__":
    app.run()